In [1]:
import numpy as np
from random import sample
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM, Embedding
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from feature_funcs import *
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import os
os.chdir("/Users/desireewaugh/Desktop/MIT/Courses/6.883 - Modeling with ML/Projects/Final Project/MusicGenerator/data")

### Prepare Data

In [42]:
def create_training3(char_nums, str_length):
    '''Create training dataset with x and y values from your numeric list.
    The x data is a list of all numeric sequences, and the y data is the next character.'''
    
    # The x_values begin at the starting indices and are str_length characters long
    # The y_values are one character after the end of the x_values
    x_data = np.array(char_nums[0:str_length])
    y_data = [char_nums[str_length]]
    for i in range(1, len(char_nums)-str_length):
        x_data = np.vstack((x_data, np.array(char_nums[i:i+str_length])))
        y_data.append(char_nums[i+str_length])
    
    #return x_data, y_data
    return x_data, y_data


In [53]:
with open("Jigs.txt") as my_file:
    abc_text = my_file.read()
    
num_to_char, char_to_num = create_dictionaries(abc_text)
vocab_length = len(num_to_char)

text_nums_train = encoder(abc_text, char_to_num)

x_vals, y_vals = create_training3(text_nums_train[0:5000], 200)
x_vals.shape

(4800, 200)

In [54]:
# https://keras.io/examples/lstm_text_generation/
# Vectorization
x = np.zeros((x_vals.shape[0], x_vals.shape[1], vocab_length))
y = np.zeros((x_vals.shape[0], vocab_length))
for i, sequence in enumerate(x_vals):
    for j, number in enumerate(sequence):
        x[i, j, number] = 1
        y[i, y_vals[i]] = 1

x.shape, y.shape

((4800, 200, 91), (4800, 91))

### Build RNN

Sources: <br />
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  <br />
https://github.com/aamini/introtodeeplearning/blob/master/lab1/Part2_Music_Generation.ipynb  <br /> https://medium.com/datadriveninvestor/music-generation-using-deep-learning-85010fb982e2 
<br /> https://keras.io/examples/lstm_text_generation/

In [15]:
model = tf.keras.Sequential()
# model.add(layers.Embedding(vocab_length, 256, batch_input_shape=x_vals.shape))
model.add(layers.LSTM(128, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(512))
model.add(layers.Dense(vocab_length, activation='softmax'))

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 200, 128)          112640    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200, 256)          394240    
_________________________________________________________________
lstm_8 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 91)                46683     
Total params: 2,128,475
Trainable params: 2,128,475
Non-trainable params: 0
_________________________________________________________________


In [17]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Fit model
model.fit(x, y, epochs=20, batch_size=50)

Train on 4800 samples
Epoch 1/20
1600/4800 [=========>....................] - ETA: 4:34 - loss: 4.0829

In [ ]:
predictions = model.predict(x)
predictions

In [44]:
text_predictions = decoder(predictions, num_to_char)
text_predictions